In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import sys
import subprocess

# 1️⃣ Uninstall any existing bitsandbytes build
#    to avoid mismatched CUDA binaries
print("🎯 Uninstalling old bitsandbytes...")
subprocess.run([sys.executable, "-m", "pip", "uninstall", "-y", "bitsandbytes"], check=True)

# 2️⃣ Install bitsandbytes >=0.43.2 which adds upstream support for CUDA 12.4+
#    (see HF docs: supports CUDA 11.0–12.5, including 12.4+) :contentReference[oaicite:0]{index=0}
print("🎯 Installing bitsandbytes>=0.43.2 for CUDA 12.4 support...")
subprocess.run([sys.executable, "-m", "pip", "install", "--upgrade", "bitsandbytes>=0.43.2"], check=True)

# 3️⃣ Ensure the CUDA 12.4 runtime path is on LD_LIBRARY_PATH
#    so bitsandbytes can locate libcudart and its own libbitsandbytes_cuda124 :contentReference[oaicite:1]{index=1}
cuda_lib="/usr/local/cuda-12.4/lib64"
prev_ld = os.environ.get("LD_LIBRARY_PATH", "")
os.environ["LD_LIBRARY_PATH"] = f"{cuda_lib}:{prev_ld}"
print(f"🎯 LD_LIBRARY_PATH set to include {cuda_lib}")

# 4️⃣ Restart prompt reminder (mandatory in Colab to pick up the new .so files)
#    In standalone scripts this is not needed, but in notebooks you must restart runtime.
print("\n⚠️  If you’re in a Colab notebook, **please restart the runtime now**\n    (Runtime → Restart runtime) and re-run this cell/script.\n")

# 5️⃣ Diagnostic: confirm bitsandbytes is seeing CUDA 12.4
print("🔍 Running bitsandbytes diagnostic...")
res = subprocess.run([sys.executable, "-m", "bitsandbytes"], capture_output=True, text=True)
print(res.stdout)
if "CUDA" not in res.stdout:
    raise RuntimeError("bitsandbytes did not detect CUDA—check your LD_LIBRARY_PATH and installation.")


🎯 Uninstalling old bitsandbytes...


🎯 Installing bitsandbytes>=0.43.2 for CUDA 12.4 support...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 84.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.9.41
  Attempting uninstall: nvidia-curand-cu12
    Found existing installation: nvidia-cu

In [3]:
!pip install --quiet --upgrade transformers peft datasets accelerate trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.1/411.1 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.1/362.1 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.0/348.0 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 12.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.


In [4]:
!pip install --quiet --upgrade torch torchvision Pillow scikit-learn evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 109.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 84.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.6/216.6 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20

In [5]:
!pip install -q transformers torch evaluate scikit-learn pillow

In [6]:
import os
import ast
import pandas as pd
from PIL import Image
import torch
from sklearn.model_selection import train_test_split  # scikit-learn split :contentReference[oaicite:2]{index=2}
import evaluate                                        # metrics :contentReference[oaicite:3]{index=3}
from transformers import AutoProcessor, AutoModelForVision2Seq , BitsAndBytesConfig, get_linear_schedule_with_warmup, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model, PeftModel, TaskType
from datasets import Dataset

CSV_PATH = "/kaggle/input/images-with-vqas/train.csv"
train_df = pd.read_csv(CSV_PATH)

train_df["image_path"] = train_df["image_path"].str.replace(
    r"^Dataset/final_dataset/",
    "/kaggle/input/images-with-vqas/final_dataset/final_dataset/",
    regex=True
)

train_df["vqa_response"] = train_df["vqa_response"].apply(ast.literal_eval)
train_df = train_df.explode("vqa_response").reset_index(drop=True)
train_df[["question", "answer"]] = pd.DataFrame(train_df["vqa_response"].tolist(), index=train_df.index)

exists = train_df["image_path"].apply(os.path.exists)
print(f"Skipping {(~exists).sum()} rows with missing images")
train_df = train_df[exists]

train_df = train_df[["image_path", "question", "answer"]].reset_index(drop=True)
print(f"Total examples: {len(train_df)}")

2025-05-16 18:26:24.982960: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747419985.177385      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747419985.232419      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Skipping 0 rows with missing images
Total examples: 29077


In [7]:
CSV_PATH = "/kaggle/input/images-with-vqas/test.csv"
test_df = pd.read_csv(CSV_PATH)

test_df["image_path"] = test_df["image_path"].str.replace(
    r"^Dataset/final_dataset/",
    "/kaggle/input/images-with-vqas/final_dataset/final_dataset/",
    regex=True
)

test_df["vqa_response"] = test_df["vqa_response"].apply(ast.literal_eval)
test_df = test_df.explode("vqa_response").reset_index(drop=True)
test_df[["question", "answer"]] = pd.DataFrame(test_df["vqa_response"].tolist(), index=test_df.index)

exists = test_df["image_path"].apply(os.path.exists)
print(f"Skipping {(~exists).sum()} rows with missing images")
test_df = test_df[exists]

test_df = test_df[["image_path", "question", "answer"]].reset_index(drop=True)
print(f"Total examples: {len(test_df)}")

Skipping 0 rows with missing images
Total examples: 29047


In [8]:
import os
import pandas as pd
from PIL import Image
from transformers import AutoProcessor, AutoModelForVisualQuestionAnswering, Trainer, TrainingArguments
import torch
from accelerate import Accelerator
from sklearn.model_selection import train_test_split
from peft import LoraConfig, get_peft_model
from transformers.data.data_collator import default_data_collator

# Initialize Accelerator for efficient multi-GPU training
# accelerator = Accelerator()

# # Load BLIP-1 VQA processor and model with fast image processing
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# processor = AutoProcessor.from_pretrained("Salesforce/blip-vqa-base", use_fast=True)
# model = AutoModelForVisualQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base").to(device)
# base_model = model



import os
import torch
from transformers import (
    Blip2ForConditionalGeneration,
    AutoProcessor,AutoModelForVisualQuestionAnswering,
    BitsAndBytesConfig,
)
from transformers import AutoProcessor, AutoModelForVision2Seq
from peft import LoraConfig, get_peft_model

# model_id = "Salesforce/blip2-flan-t5-xl"
model_id = "Salesforce/blip2-opt-2.7b"
# model_id ="ibm-granite/granite-vision-3.2-2b"

# 1) Quantization config
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
)
# quantization_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     # compute in float32 so all dequantization goes to float32
#     bnb_4bit_compute_dtype=torch.float32,
#     bnb_4bit_quant_type="nf4",
# )

# 2) Load base model & processor
base_model = AutoModelForVision2Seq.from_pretrained(
    model_id,
    quantization_config=quantization_config,
    device_map="auto",
)
processor = AutoProcessor.from_pretrained(model_id)
if processor.tokenizer.pad_token is None:
    processor.tokenizer.pad_token = processor.tokenizer.eos_token

# 3) LoRA config and wrap
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
     target_modules=["query", "value"],
    lora_dropout=0.05,
    bias="none",
    # task_type="CAUSAL_LM",
)  # :contentReference[oaicite:0]{index=0}

model = get_peft_model(base_model, lora_config)  # wraps base_model in-place :contentReference[oaicite:1]{index=1}

config.json:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/122k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/10.0G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.56M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/548 [00:00<?, ?B/s]

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [10]:
from torch.utils.data import DataLoader
class VQADataset(torch.utils.data.Dataset):
    def __init__(self, df, processor, max_length=32):
        self.df = df
        self.processor = processor
        self.max_length = max_length
        

    def __len__(self):
        return len(self.df)
        
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image = Image.open(row.image_path).convert("RGB")
        question, answer = row.question, row.answer
        
        encoding = self.processor(
            images=image, 
            text=question,
            padding="max_length", truncation=True,
            max_length=self.max_length, 
            return_tensors="pt",
            return_attention_mask=True
        )
    
        encoding = {k: v.squeeze(0) for k, v in encoding.items()}
        encoding["labels"] = self.processor.tokenizer(
            answer,
            return_tensors="pt",
            padding="max_length",
            truncation=True,
            max_length=self.max_length
        )["input_ids"].squeeze(0)

        return encoding

In [11]:
import time
from transformers import ProgressCallback

# Prepare model with Accelerator
# model = accelerator.prepare(model)

# Define training arguments
training_args = TrainingArguments(
    output_dir="/kaggle/working/blip2_finetuned",
    run_name="blip2_lora_finetune",  # Unique run name
    num_train_epochs=3,
    per_device_train_batch_size=4,  # Reduced for memory stability
    gradient_accumulation_steps=4,  # Simulate larger batch size (effective batch size = 16)
    learning_rate=5e-5,
    weight_decay=0.01,
    logging_dir='/kaggle/working/logs',
    logging_steps=10,
    save_strategy="epoch",
    fp16=True,  # Mixed precision for efficiency
    remove_unused_columns=False,  # Keep all dataset columns
    report_to="none"  # Disable W&B and other logging integrations
)

# Create Trainer instance with default data collator
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=default_data_collator,  # Handle tensor stacking
    callbacks=[ProgressCallback]
)

# Check GPU memory usage before training
# if torch.cuda.is_available():
#     print("GPU Memory Usage Before Training:")
#     print(torch.cuda.memory_summary())

CHUNK_SIZE = 3000
n_rows     = len(train_df)
total_start = time.time()
count =0
for start in range(0, n_rows, CHUNK_SIZE):
    end   = min(start + CHUNK_SIZE, n_rows)
    batch = train_df.iloc[start:end]                          # grab rows [start:end)

    # — example “prepare” step; use your real preprocessing here:
    
    train_ds = VQADataset(batch, processor)
    trainer.train_dataset = train_ds

    if time.time() - total_start >= 28800:
        break
    
    start_time = time.time()
    print("Starting training...")
    # — train on just this chunk:
    trainer.train()
    elapsed = (time.time() - start_time) / 60
    print(f"Training completed in {elapsed:.2f} minutes.")

    print(f"Trained on rows {start}–{end-1} ({end-start} examples).")
    count +=1
    # if count == 5:
    #     break

trainer.save_model("/kaggle/working/blip2_finetuned")
print("Model saved to '/kaggle/working/blip2_finetuned'")

No label_names provided for model class `PeftModel`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Starting training...


  0%|          | 0/561 [00:00<?, ?it/s]

Step,Training Loss
10,17.117300
20,16.995500
30,16.922200
40,16.741400
50,16.529100
60,16.252100
70,15.856800
80,15.659100
90,15.062300
100,14.466300


{'loss': 17.1173, 'grad_norm': 1.2134616374969482, 'learning_rate': 4.919786096256685e-05, 'epoch': 0.05}
{'loss': 16.9955, 'grad_norm': 1.2055765390396118, 'learning_rate': 4.848484848484849e-05, 'epoch': 0.11}
{'loss': 16.9222, 'grad_norm': 1.5081443786621094, 'learning_rate': 4.759358288770054e-05, 'epoch': 0.16}
{'loss': 16.7414, 'grad_norm': 1.8053431510925293, 'learning_rate': 4.679144385026738e-05, 'epoch': 0.21}
{'loss': 16.5291, 'grad_norm': 2.7878448963165283, 'learning_rate': 4.590017825311943e-05, 'epoch': 0.27}
{'loss': 16.2521, 'grad_norm': 2.5227115154266357, 'learning_rate': 4.500891265597148e-05, 'epoch': 0.32}
{'loss': 15.8568, 'grad_norm': 2.492065906524658, 'learning_rate': 4.411764705882353e-05, 'epoch': 0.37}
{'loss': 15.6591, 'grad_norm': 3.522611141204834, 'learning_rate': 4.322638146167558e-05, 'epoch': 0.43}
{'loss': 15.0623, 'grad_norm': 3.8908069133758545, 'learning_rate': 4.233511586452763e-05, 'epoch': 0.48}
{'loss': 14.4663, 'grad_norm': 9.026497840881348

...

Step,Training Loss
10,0.659000
20,0.658600
30,0.660000
40,0.684100
50,0.652500
60,0.654400
70,0.673400
80,0.635900
90,0.676800
100,0.640100


{'loss': 0.659, 'grad_norm': 2.221527338027954, 'learning_rate': 4.919786096256685e-05, 'epoch': 0.05}
{'loss': 0.6586, 'grad_norm': 2.213428258895874, 'learning_rate': 4.83065953654189e-05, 'epoch': 0.11}
{'loss': 0.66, 'grad_norm': 3.1991426944732666, 'learning_rate': 4.741532976827095e-05, 'epoch': 0.16}
{'loss': 0.6841, 'grad_norm': 1.9877815246582031, 'learning_rate': 4.6524064171123e-05, 'epoch': 0.21}
{'loss': 0.6525, 'grad_norm': 2.390392303466797, 'learning_rate': 4.563279857397505e-05, 'epoch': 0.27}
{'loss': 0.6544, 'grad_norm': 2.1423614025115967, 'learning_rate': 4.474153297682709e-05, 'epoch': 0.32}
{'loss': 0.6734, 'grad_norm': 2.6925852298736572, 'learning_rate': 4.385026737967914e-05, 'epoch': 0.37}
{'loss': 0.6359, 'grad_norm': 2.1916253566741943, 'learning_rate': 4.2959001782531194e-05, 'epoch': 0.43}
{'loss': 0.6768, 'grad_norm': 2.135016679763794, 'learning_rate': 4.2067736185383246e-05, 'epoch': 0.48}
{'loss': 0.6401, 'grad_norm': 2.2526845932006836, 'learning_rat

  0%|          | 0/390 [00:00<?, ?it/s]

Step,Training Loss
10,0.617500
20,0.640400
30,0.655900
40,0.649300
50,0.660800
60,0.609300
70,0.647600
80,0.653000
90,0.629800
100,0.605700


{'loss': 0.6175, 'grad_norm': 2.1618473529815674, 'learning_rate': 4.884615384615385e-05, 'epoch': 0.08}
{'loss': 0.6404, 'grad_norm': 2.7657320499420166, 'learning_rate': 4.7564102564102563e-05, 'epoch': 0.15}
{'loss': 0.6559, 'grad_norm': 2.097031354904175, 'learning_rate': 4.6282051282051287e-05, 'epoch': 0.23}
{'loss': 0.6493, 'grad_norm': 2.078153133392334, 'learning_rate': 4.5e-05, 'epoch': 0.31}
{'loss': 0.6608, 'grad_norm': 2.4255893230438232, 'learning_rate': 4.371794871794872e-05, 'epoch': 0.38}
{'loss': 0.6093, 'grad_norm': 2.2222940921783447, 'learning_rate': 4.2435897435897435e-05, 'epoch': 0.46}
{'loss': 0.6476, 'grad_norm': 2.4938464164733887, 'learning_rate': 4.115384615384615e-05, 'epoch': 0.54}
{'loss': 0.653, 'grad_norm': 2.328542947769165, 'learning_rate': 3.9871794871794875e-05, 'epoch': 0.62}
{'loss': 0.6298, 'grad_norm': 2.23734188079834, 'learning_rate': 3.858974358974359e-05, 'epoch': 0.69}
{'loss': 0.6057, 'grad_norm': 2.2864532470703125, 'learning_rate': 3.73

In [12]:
# from peft import PeftModel
# from transformers import Blip2ForConditionalGeneration, Blip2Processor

# # Load the base BLIP-2 model
# model_id = "Salesforce/blip2-opt-2.7b"  # Replace with your model ID
# base_model = Blip2ForConditionalGeneration.from_pretrained(
#     model_id,
#     device_map="auto"  # Adjust based on your setup
# )

# Load the LoRA adapters on top of the base model
tuned_model = PeftModel.from_pretrained(model, "/kaggle/working/blip2_finetuned")

# Set the model to evaluation mode
tuned_model.eval()

# # Load the processor (same as used in training)
# processor = Blip2Processor.from_pretrained(model_id)

/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:167: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/peft_model.py:569: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.base_model.model.qformer.encoder.layer.0.attention.attention.query.lora_A.default.weight', 'base_model.model.base_model.model.qformer.encoder.layer.0.attention.attention.query.lora_B.default.weight', 'base_model.model.base_model.model.qformer.encoder.layer.0.attention.attention.value.lora_A.default.weight', 'base_model.model.base_model.model.qformer.encoder.layer.0.attention.attention.value.lora_B.default.weight', 'base_model.model.base_model.model.qformer.encoder.layer.0.crossattention.attention.query.lora_A.default.weight', 'base_model.model.base_model.model.qformer.encoder.layer.0.cros

PeftModel(
  (base_model): LoraModel(
    (model): PeftModel(
      (base_model): LoraModel(
        (model): Blip2ForConditionalGeneration(
          (vision_model): Blip2VisionModel(
            (embeddings): Blip2VisionEmbeddings(
              (patch_embedding): Conv2d(3, 1408, kernel_size=(14, 14), stride=(14, 14))
            )
            (encoder): Blip2Encoder(
              (layers): ModuleList(
                (0-38): 39 x Blip2EncoderLayer(
                  (self_attn): Blip2Attention(
                    (dropout): Dropout(p=0.0, inplace=False)
                    (qkv): Linear4bit(in_features=1408, out_features=4224, bias=True)
                    (projection): Linear4bit(in_features=1408, out_features=1408, bias=True)
                  )
                  (layer_norm1): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
                  (mlp): Blip2MLP(
                    (activation_fn): GELUActivation()
                    (fc1): Linear4bit(in_features=1408, out

In [13]:
from tqdm import tqdm
import torch
from PIL import Image

# Assuming 'test_df' has columns 'image_path', 'question', 'answer'
# and 'device' is defined (e.g., device = torch.device("cuda"))

preds, refs = [], []
count = 0
for row in tqdm(test_df.itertuples(), total=len(test_df)):
    img = Image.open(row.image_path).convert("RGB")
    prompt = f"Based on the image, answer the following question with a single word. Question: {row.question} Answer:"
    inputs = processor(images=img, text=prompt, return_tensors="pt").to(device)
    pixel_values = inputs["pixel_values"]
    input_ids = inputs["input_ids"]
    attention_mask = inputs["attention_mask"]
    
    # Generate prediction with the fine-tuned model
    with torch.no_grad():
        out = tuned_model.generate(
            pixel_values=pixel_values,
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_new_tokens=10
        )
    
    text = processor.batch_decode(out, skip_special_tokens=True)[0].strip()
    # print("Generated text:", text)
    
    # Extract the last word as the prediction
    pred = text.split()[-1].rstrip(".,;:!?") if text else ""
    preds.append(pred.lower())
    refs.append(row.answer.lower())
    
    # print("Prediction:", pred.lower())
    # print("Reference:", row.answer.lower())
    
    count += 1
    # if count == 100:
    #     break

100%|██████████| 29047/29047 [3:30:52<00:00,  2.30it/s]


In [14]:
# ──────────────────────────────────────────────────────────────────────────────
# 6. Compute Metrics (and identify mismatches)
# ──────────────────────────────────────────────────────────────────────────────
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import evaluate
# from bert_score import BERTScorer # Keep if you add BERTScore later
# from rouge_score import rouge_scorer # Keep if you add ROUGE later
# from rapidfuzz.distance import Levenshtein # Keep if you add Levenshtein later
from sentence_transformers import SentenceTransformer, util # Keep if you add SBERT later
import numpy as np
import pandas as pd # Ensure pandas is imported if df is used



# normalize case
preds_l = [str(p).lower().strip() for p in preds] # Added str() and strip() for robustness
refs_l  = [str(r).lower().strip() for r in refs]   # Added str() and strip()

# # 1) TOKEN-LEVEL EXACT MATCH → binary labels
y_pred_bin = [int(p == r) for p, r in zip(preds_l, refs_l)]
y_true_bin = [1]*len(refs_l)            # reference is always “correct” class

acc = accuracy_score(y_true_bin, y_pred_bin)
prec, rec, f1, _ = precision_recall_fscore_support(
    y_true_bin, y_pred_bin, average="binary", zero_division=0
)
print(f"Exact-match Accuracy: {acc:.3f}")
print(f"Exact-match Precision: {prec:.3f}") # Will be 1.0 if acc > 0, else 0.0
print(f"Exact-match Recall:    {rec:.3f}")    # Will be same as acc
print(f"Exact-match F1:        {f1:.3f}\n")  

Exact-match Accuracy: 0.508
Exact-match Precision: 1.000
Exact-match Recall:    0.508
Exact-match F1:        0.673



In [15]:
! pip install git+https://github.com/google-research/bleurt.git
!pip install scikit-learn evaluate bert-score rouge-score \
            sentence-transformers rapidfuzz


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Cloning https://github.com/google-research/bleurt.git to /tmp/pip-req-build-_t7i0vpl
  Running command git clone --filter=blob:none --quiet https://github.com/google-research/bleurt.git /tmp/pip-req-build-_t7i0vpl
  Resolved https://github.com/google-research/bleurt.git to commit cebe7e6f996b40910cfaa520a63db47807e3bf5c
  Preparing metadata (setup.py) ... done
  Created wheel for BLEURT: filename=BLEURT-0.0.2-py3-none-any.whl size=16456766 sha256=47536355dfaa86986031051c730036089d4ab053bade235333fe5f4c1b01ccc3
  Stored in directory: /tmp/pip-ephem-wheel-cache-twooxjt6/wheels/30/af/34/e148007788b060e4c76e7ecf68e70c692dff0f2632e62ac454
Successfully built BLEURT


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 37.3 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=e6683598344e61e22713394a3b23d7007119fe65a7c2616a5060197313860fa8
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score


In [16]:
from bert_score import BERTScorer # Keep if you add BERTScore later
from rouge_score import rouge_scorer # Keep if you add ROUGE later
from rapidfuzz.distance import Levenshtein # Keep if you add Levenshtein later
from sentence_transformers import SentenceTransformer, util # Keep if you add SBERT later

In [17]:
# --- ROUGE ---
print("\n--- ROUGE Scores ---")
rouge_eval_scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
rouge1_scores = []
rougeL_scores = []
for pred, ref in zip(preds_l, refs_l):
    if not pred or not ref: # Handle empty strings if any
        rouge1_scores.append(0.0)
        rougeL_scores.append(0.0)
        continue
    scores = rouge_eval_scorer.score(ref, pred) # Target, Prediction
    rouge1_scores.append(scores['rouge1'].fmeasure)
    rougeL_scores.append(scores['rougeL'].fmeasure)

if rouge1_scores:
    print(f"Average ROUGE-1 F1: {np.mean(rouge1_scores):.3f}")
    print(f"Average ROUGE-L F1: {np.mean(rougeL_scores):.3f}")
else:
    print("No ROUGE scores to compute (empty predictions or references).")


# --- BERTScore ---
print("\n--- BERTScore ---")
try:
    bert_eval_scorer = BERTScorer(lang="en", rescale_with_baseline=True, device=device)
    filtered_preds_l = [p for p, r in zip(preds_l, refs_l) if p and r]
    filtered_refs_l = [r for p, r in zip(preds_l, refs_l) if p and r]

    if filtered_preds_l and filtered_refs_l:
        P, R, F1_bert = bert_eval_scorer.score(filtered_preds_l, filtered_refs_l)
        print(f"Average BERTScore Precision: {P.mean():.3f}")
        print(f"Average BERTScore Recall:    {R.mean():.3f}")
        print(f"Average BERTScore F1:        {F1_bert.mean():.3f}")
    else:
        print("Not enough valid (non-empty) prediction/reference pairs for BERTScore.")
except Exception as e:
    print(f"Could not compute BERTScore: {e}")


# --- Levenshtein Normalized Similarity ---
print("\n--- Levenshtein Normalized Similarity ---")
lev_similarities = []
for pred, ref in zip(preds_l, refs_l):
    if not pred and not ref: # both empty, perfect match
        similarity = 1.0
    elif not pred or not ref: # one empty, other not, zero similarity
        similarity = 0.0
    else:
        similarity = Levenshtein.normalized_similarity(pred, ref)
    lev_similarities.append(similarity)

if lev_similarities:
    print(f"Average Levenshtein Normalized Similarity: {np.mean(lev_similarities):.3f}")
else:
    print("No Levenshtein similarities to compute.")


# --- Sentence Transformer Cosine Similarity ---
print("\n--- Sentence-BERT Cosine Similarity ---")
try:
    sbert_model = SentenceTransformer('all-MiniLM-L6-v2', device=device)
    valid_pairs_indices = [i for i, (p, r) in enumerate(zip(preds_l, refs_l)) if p and r]
    sbert_preds = [preds_l[i] for i in valid_pairs_indices]
    sbert_refs = [refs_l[i] for i in valid_pairs_indices]

    if sbert_preds and sbert_refs:
        embeddings_preds = sbert_model.encode(sbert_preds, convert_to_tensor=True)
        embeddings_refs = sbert_model.encode(sbert_refs, convert_to_tensor=True)

        cosine_scores_sbert = util.cos_sim(embeddings_preds, embeddings_refs)
        pair_similarities = [cosine_scores_sbert[i, i].item() for i in range(len(sbert_preds))]
        print(f"Average Sentence-BERT Cosine Similarity: {np.mean(pair_similarities):.3f}")
    else:
        print("Not enough valid (non-empty) prediction/reference pairs for Sentence-BERT similarity.")
except Exception as e:
    print(f"Could not compute Sentence-BERT similarity: {e}")



--- ROUGE Scores ---
Average ROUGE-1 F1: 0.515
Average ROUGE-L F1: 0.515

--- BERTScore ---


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/908 [00:00<?, ?it/s]

Batches:   0%|          | 0/908 [00:00<?, ?it/s]

Average Sentence-BERT Cosine Similarity: 0.755


In [18]:
!pip freeze > requirements.txt

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
